In [1]:
import msprime as msp
import fpop
import importlib; importlib.reload(fpop)
import numpy as np
from IPython.display import SVG

In [16]:
sim = msp.simulate(sample_size=20, length=1000000, mutation_rate=1e-4, 
                   recombination_rate=1e-5, Ne=1, random_seed=1111)

## Ground truth:

In [17]:
SVG(sim.draw_svg())

KeyboardInterrupt: 

In [ ]:
print(sim.draw_text())

## New method
The new method decodes ARGs using some ideas from changepoint detection. The `fpop.decode` method does all the work:

In [12]:
?fpop.decode

Signature: fpop.decode(L: int, panel: numpy.ndarray, positions: numpy.ndarray, theta: Union[float, Sequence[float]], rho: float, w: int = 100) -> tskit.trees.TreeSequence
Docstring:
Estimate an ARG by iteratively threading haplotypes using optimal
partitioning algorithm.

Args:
    L: overall length of sequence data.
    panel: Binary array of shape [n, P] encoding the allelic state of each of
        n haplotypes at P marker positions.
    positions: Integer array of shape [P] giving the zero-based position of
        each marker. It must be true that max(positions) < L.
    w: Aggregate the data into buckets of size :w: base pairs, and multiply
        :rho: and :theta: by :w:. Setting :w:=1 is exact but slow. :w: must
        evenly divide :L:.
    theta: Array of dimension [L] (or something broadcastable to
        that) giving the scaled mutation rate at each position.
    rho: The scaled recombination rate between each position.

Returns:
    A tree sequence representing the deco

In [19]:
positions = np.array([int(v.position) for v in sim.variants()])
panel = np.transpose([v.genotypes for v in sim.variants()])
estimated_ts = fpop.decode(L=1000000, panel=panel, positions=positions, theta=1e-4, rho=1e-5, w=100)

DEBUG:fpop.decode:iteration i=1 bt=[{'hap': 0, 'pos': 0, 'm': {'f': 147.98495785924567, 'x': -0.0020020026707190616}}, {'hap': 0, 'pos': 2268, 'm': {'f': 1415.868419314651, 'x': -1.376862471498372}}, {'hap': 0, 'pos': 9014, 'm': {'f': 1475.9167579161328, 'x': 0.16940516486070994}}]
DEBUG:fpop.decode:threading lineage i=1 spans=[2268 6746  986] times=[0.501002  1.9812249 0.4220834] haps=(0, 0, 0)
DEBUG:fpop.decode:iteration i=2 bt=[{'hap': 0, 'pos': 0, 'm': {'f': 54.522852523329675, 'x': 1.0090066403738553}}, {'hap': 0, 'pos': 1813, 'm': {'f': 228.92864568729, 'x': -1.8897558623707398}}, {'hap': 0, 'pos': 2354, 'm': {'f': 362.9751200581718, 'x': 0.4146951075777251}}, {'hap': 1, 'pos': 5200, 'm': {'f': 385.2558673265335, 'x': -0.43734284700618464}}, {'hap': 0, 'pos': 5253, 'm': {'f': 416.8742869661129, 'x': 0.3966261594409681}}, {'hap': 1, 'pos': 5747, 'm': {'f': 735.5738097326812, 'x': -0.5019937718981675}}, {'hap': 1, 'pos': 8977, 'm': {'f': 775.5834541825462, 'x': 0.8625523454206553}}

## Truth vs. estimated

In [ ]:
SVG(sim.draw_svg())

In [ ]:
SVG(estimated_ts.draw_svg())

In [9]:
print(sim.draw_text())

4.01┊                     ┊                     ┊                     ┊                     ┊      23             ┊                     ┊    
    ┊                     ┊                     ┊                     ┊                     ┊  ┏━━━━┻━━━┓         ┊                     ┊    
2.83┊                     ┊                     ┊                     ┊                     ┊  ┃        ┃         ┊      22             ┊    
    ┊                     ┊                     ┊                     ┊                     ┊  ┃        ┃         ┊  ┏━━━━┻━━━┓         ┊    
2.11┊       21            ┊      21             ┊                     ┊                     ┊  ┃        ┃         ┊  ┃        ┃         ┊    
    ┊   ┏━━━━┻━━━━┓       ┊  ┏━━━━┻━━━┓         ┊                     ┊                     ┊  ┃        ┃         ┊  ┃        ┃         ┊    
1.47┊   ┃         ┃       ┊  ┃        ┃         ┊                     ┊      20             ┊  ┃        ┃         ┊  ┃        ┃         ┊    
    ┊ 

In [10]:
print(estimated_ts.draw_text())

1.66┊       49            ┊                     ┊                     ┊                     ┊                     ┊                     ┊                     ┊                     ┊    
    ┊   ┏━━━━┻━━━━┓       ┊                     ┊                     ┊                     ┊                     ┊                     ┊                     ┊                     ┊    
1.31┊   ┃         ┃       ┊                     ┊      47             ┊      48             ┊        48           ┊          48         ┊        48           ┊           48        ┊    
    ┊   ┃         ┃       ┊                     ┊  ┏━━━━┻━━━┓         ┊  ┏━━━━┻━━━┓         ┊   ┏━━━━━┻━━━━━┓     ┊    ┏━━━━━━┻━━━━━━┓  ┊   ┏━━━━━┻━━━━━┓     ┊      ┏━━━━━┻━━━━━┓  ┊    
0.83┊   ┃        45       ┊       46            ┊  ┃       43         ┊  ┃       44         ┊   ┃           ┃     ┊    ┃             ┃  ┊   ┃           ┃     ┊      ┃           ┃  ┊    
    ┊   ┃     ┏━━━┻━━━┓   ┊   ┏━━━━┻━━━━┓       ┊  ┃    ┏━━━┻━━━━┓    